In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
cmap = plt.cm.get_cmap('coolwarm')
import warnings
warnings.filterwarnings('ignore')


# Use for pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from xgboost import XGBRegressor
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/tmp/ipykernel_33/1794285695.py:11: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('coolwarm')


/kaggle/input/test-processed-csv/test_processed.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv
/kaggle/input/train-processed-csv/train_processed.csv


In [12]:
df_train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
df_test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')

In [13]:
essay_id_dropped = df_train['essay_id']
df_train = df_train.drop('essay_id', axis = 1)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  17307 non-null  object
 1   score      17307 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 270.5+ KB


In [14]:
# Number of words
def preprocess_df(df):
    df['word_count'] = df['full_text'].apply(lambda x: len(x.split()))

    # Length
    df['essay_length'] = df['full_text'].str.len()

    # Sentences count
    # Adding a new column 'sentences_count' that counts the sentences in 'full_text'
    df['sentences_count'] = df['full_text'].str.count(r'\.')

    # Paragraph count
    # Adding a new column 'paragraph_count' that counts the paragraphs in 'full_text'
    df['paragraph_count'] = df['full_text'].str.count(r'\n') + 1
    
    return df

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    x = x.lower()
    x = removeHTML(x)
    x = re.sub("@\w+", '',x)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub("http\w+", '',x)
    x = re.sub(r"\s+", " ", x)
#     x = expandContractions(x)
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = x.strip()
    return x

In [15]:
df_train = preprocess_df(df_train)
df_train.head(5)

,full_text,score,word_count,essay_length,sentences_count,paragraph_count
0,Many people have car where they live. The thin...,3,498,2677,13,1
1,I am a scientist at NASA that is discussing th...,3,332,1669,19,9
2,People always wish they had the same technolog...,4,550,3077,24,7
3,"We all heard about Venus, the planet without a...",4,451,2701,23,9
4,"Dear, State Senator\n\nThis is a letter to arg...",3,373,2208,15,11


In [16]:
# Tìm các dòng có giá trị LotFrontage lớn hơn 300 và lấy index của chúng

rows_to_drop = df_train.query('sentences_count > 60 | (sentences_count > 50 & score == 1)').index

# Xóa các dòng có index tương ứng

df_train.drop(rows_to_drop, inplace=True)

In [17]:
# Tìm các dòng có giá trị LotFrontage lớn hơn 300 và lấy index của chúng

rows_to_drop = df_train.query('paragraph_count > 80 | (paragraph_count > 55 & score == 1 ) | (paragraph_count >60 & score == 2 ) | (paragraph_count > 30  & score == 5) | (paragraph_count > 25  & score == 6)').index

# Xóa các dòng có index tương ứng

df_train.drop(rows_to_drop, inplace=True)

In [18]:
# Define transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))
])

# Update categorical and numerical columns
numerical_columns = df_train.select_dtypes('int64').columns
categorical_columns = df_train.select_dtypes('object').columns

# Remove target variable from numerical columns
numerical_columns = numerical_columns.drop('score')

# Combine transformers using ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],remainder = 'passthrough')

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)])

# Apply the pipeline to your dataset
X = df_train.drop('score', axis=1)
y = np.log(df_train['score']) #normalize dependent variable 
X_preprocessed = pipeline.fit_transform(X)

In [79]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# # Chuyển đổi văn bản thành đặc trưng sử dụng TF-IDF
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(df['full_text'])
# y = df['score'] - 1

In [19]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, 
                                                    test_size=0.2, random_state=42)

In [10]:
# Define the models
models = {
    'XGBoost': XGBRegressor(random_state=42),
}

# Define the hyperparameter grids for each model
param_grids = {
    'XGBoost': {
        'n_estimators': [100],
        'learning_rate': [0.1],
        'max_depth': [3],
        'subsample': [1.0],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
for model_name, model in models.items():
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best RMSE for XGBoost: 0.2943404135826672



In [21]:
# df_test = pd.read_csv('/kaggle/input/test-processed-csv/test_processed.csv')
# df_test

df_test = preprocess_df(df_test)
df_test.head()

,essay_id,full_text,word_count,essay_length,sentences_count,paragraph_count
0,000d118,Many people have car where they live. The thin...,498,2677,13,1
1,000fe60,I am a scientist at NASA that is discussing th...,332,1669,19,9
2,001ab80,People always wish they had the same technolog...,550,3077,24,7


In [22]:
# Apply the pipeline to dataset
X_test_preprocessed = pipeline.transform(df_test)

# Lấy mô hình tốt nhất
best_model = grids['XGBoost'].best_estimator_
best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [23]:
# Lưu lại cột essay_id để sử dụng sau này
essay_ids = df_test['essay_id']

# Xóa cột essay_id trước khi tiền xử lý
X_test = df_test.drop('essay_id', axis=1)

# Áp dụng pipeline đã huấn luyện để tiền xử lý dữ liệu test
X_test_preprocessed = pipeline.transform(X_test)

# Dự đoán điểm số trên dữ liệu test đã tiền xử lý
y_pred_log = grids['XGBoost'].predict(X_test_preprocessed)

# Chuyển đổi ngược từ log-transform
y_pred = np.exp(y_pred_log)

# Tạo dataframe chứa essay_id và dự đoán điểm số
results = pd.DataFrame({'essay_id': essay_ids, 'score': y_pred})

# Làm tròn điểm số
results['score'] = round(results['score'])
results['score'] = results['score'].astype(int)

# Lưu kết quả ra file csv
submission = results.to_csv('submission.csv', index=False)

In [24]:
results

,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


[CV] END learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time= 1.7min
[CV] END learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time= 1.7min
[CV] END learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time= 1.9min
